In [1]:
import os
dirname = "../data/parse/markdown_v2"

os.listdir(dirname)

['HD quy trình phát triển Đảng viên.md',
 'QC Dân chủ.md',
 'QC quản lý sử dụng tài sản ĐHBKHN.md',
 'QC Quản lý tài chính.md',
 'QC tuyển sinh ĐH 2023.md',
 'QC Tổ chức và hoạt động.md',
 'QtĐ ban hành quy chế đào tạo 2023.md',
 'QĐ học bổng gắn kết quê hương.md',
 'QĐ Học bổng KKHT 2023.md',
 'QĐ Học bổng Trần Đại Nghĩa 2023.md',
 'QĐ miễn giảm học phí 2023.md',
 'QĐ Quản lý câu lạc bộ sinh viên 2023.md',
 'QĐ quản lý hoạt động hợp tác đối ngoại.md',
 'QĐ quản lý sinh viên nước ngoài 2023.md',
 'QĐ thi Olympic và ĐMST 2023.md',
 'QĐ tổ chức thi tiếng Anh nội bộ.md',
 'QĐ về đánh giá cán bộ LĐQL.md',
 'QĐ xét cấp HB tài trợ 2024.md',
 'QĐ đánh giá các TCCSĐ trực thuộc Đảng Ủy khối.md',
 'QĐ đánh giá điểm rèn luyện sinh viên 2023.md']

In [2]:
path = os.path.join(dirname, os.listdir(dirname)[13])
with open(path, "r") as f:
    content = f.read()


In [3]:
import re
from typing import List, Dict, Any, Optional

def extract_articles(text: str) -> List[Dict[str, Any]]:

    # Pattern to match articles
    article_pattern = re.compile(
        r'#\s*(?:ĐIỀU|Điều)\s+(\d+)(?:\.|\s*\.)?\s*([^\n]+)?\n(.*?)(?=(?:#\s*(?:ĐIỀU|Điều|CHƯƠNG|Chương|MỤC|Mục)|\Z))',
        re.DOTALL | re.IGNORECASE
    )
    
    # Pattern for numbered paragraphs
    paragraph_pattern = re.compile(r'^\s*(\d+)[\.:]\s*(.+?)(?=\n\s*\d+[\.:]\s*|\Z)', re.DOTALL | re.MULTILINE)
    
    articles = []
    for match in article_pattern.finditer(text):
        article_num = match.group(1)
        article_title = match.group(2).strip() if match.group(2) else ""
        article_content = match.group(3).strip()
        
        # Extract paragraphs
        paragraphs = {}
        for para_match in paragraph_pattern.finditer(article_content):
            para_num = para_match.group(1)
            para_content = para_match.group(2).strip()
            paragraphs[para_num] = para_content
        
        articles.append({
            "number": article_num,
            "title": article_title,
            "content": article_content,
            "paragraphs": paragraphs
        })
    
    return articles

def extract_sections(text: str) -> List[Dict[str, Any]]:

    # Pattern to match sections
    section_pattern = re.compile(
        r'#\s*(?:MỤC|Mục)\s+([IVXLCDM0-9]+)(?:[\.:]?\s*([^\n]+))?',
        re.IGNORECASE
    )
    
    sections = []
    section_matches = list(section_pattern.finditer(text))
    
    for i, match in enumerate(section_matches):
        section_num = match.group(1)
        section_title = match.group(2).strip() if match.group(2) else ""
        
        # Determine section content range
        start_pos = match.end()
        end_pos = len(text)
        if i < len(section_matches) - 1:
            end_pos = section_matches[i + 1].start()
        
        section_content = text[start_pos:end_pos]
        
        # Extract articles in this section
        section_articles = extract_articles(section_content)
        
        sections.append({
            "number": section_num,
            "title": section_title,
            "articles": section_articles
        })
    
    return sections

def extract_chapters(text: str) -> List[Dict[str, Any]]:

    # Pattern to match chapters
    chapter_pattern = re.compile(
        r'#\s*(?:(?:CHƯƠNG|Chương)\s+([IVXLCDM]+)|([IVXLCDM]+)(?:-|\s+-\s*))(?:\s*(.+?))?(?=\n|$)',
        
        re.IGNORECASE
    )
    
    chapters = []
    chapter_matches = list(chapter_pattern.finditer(text))
    
    for i, match in enumerate(chapter_matches):
        chapter_num = match.group(1) if match.group(1) else match.group(2)
        chapter_title = match.group(3).strip() if match.group(3) else ""
        
        # Check for title on next line if not in heading
        if not chapter_title:
            title_pattern = re.compile(r'#\s*([^\n#]+)')
            next_line = text[match.end():].lstrip()
            title_match = title_pattern.match(next_line)
            if title_match:
                chapter_title = title_match.group(1).strip()
        
        # Determine chapter content range
        start_pos = match.end()
        end_pos = len(text)
        if i < len(chapter_matches) - 1:
            end_pos = chapter_matches[i + 1].start()
        
        chapter_content = text[start_pos:end_pos]
        
        # Extract sections in this chapter
        chapter_sections = extract_sections(chapter_content)
        
        # If no sections, extract articles directly
        chapter_articles = []
        if not chapter_sections:
            chapter_articles = extract_articles(chapter_content)
        
        chapters.append({
            "number": chapter_num,
            "title": chapter_title,
            "sections": chapter_sections,
            "articles": chapter_articles
        })
    
    return chapters

def parse_document(text: str) -> Dict[str, Any]:

    # Try to extract chapters
    chapters = extract_chapters(text)
    
    # If no chapters, try sections
    sections = []
    articles = []
    if not chapters:
        sections = extract_sections(text)
        
        # If no sections either, extract articles directly
        if not sections:
            articles = extract_articles(text)
    
    return {
        "chapters": chapters,
        "sections": sections,
        "articles": articles
    }

def get_chunks_with_metadata(text: str) -> List[Dict[str, Any]]:

    parsed_doc = parse_document(text)
    chunks = []
    
    # Process articles directly in document
    for article in parsed_doc["articles"]:
        chunks.append({
            "text": article["content"],
            "metadata": {
                "article_number": article["number"],
                "article_title": article["title"],
                "chapter_number": None,
                "chapter_title": None,
                "section_number": None,
                "section_title": None
            }
        })
    
    
    # Process articles in chapters
    for chapter in parsed_doc["chapters"]:
        # Articles directly in chapter
        for article in chapter["articles"]:
            chunks.append({
                "text": article["content"],
                "metadata": {
                    "article_number": article["number"],
                    "article_title": article["title"],
                    "chapter_number": chapter["number"],
                    "chapter_title": chapter["title"],
                    "section_number": None,
                    "section_title": None
                }
            })
        
        # Articles in sections within chapters
        for section in chapter["sections"]:
            for article in section["articles"]:
                chunks.append({
                    "text": article["content"],
                    "metadata": {
                        "article_number": article["number"],
                        "article_title": article["title"],
                        "chapter_number": chapter["number"],
                        "chapter_title": chapter["title"],
                        "section_number": section["number"],
                        "section_title": section["title"]
                    }
                })
    
    return chunks


chunks = get_chunks_with_metadata(content)

# Print result
import json
print(json.dumps(chunks, ensure_ascii=False, indent=2))

[
  {
    "text": "1. # Phạm vi điều chỉnh\n\n1. Quy định này quy định về trách nhiệm của các đơn vị, cá nhân có liên quan trong Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội hoặc Đại học) đối với công tác quản lý và hỗ trợ người nước ngoài (là người không mang quốc tịch Việt Nam hoặc là người Việt Nam mang hộ chiếu nước ngoài) đến học tập, thực tập, nghiên cứu, trao đổi tại ĐHBK Hà Nội;\n2. Quy định này quy định về quyền lợi và trách nhiệm của người nước ngoài trong quá trình học tập, thực tập, tham quan, trao đổi tại ĐHBK Hà Nội.\n2. # Đối tượng áp dụng\n\nQuy định này áp dụng đối với người nước ngoài đến học tập, thực tập, nghiên cứu, trao đổi tại ĐHBK Hà Nội (sau đây gọi chung là Lưu học sinh và viết tắt là LHS) và được chia thành các nhóm như sau:\n\n1. Lưu học sinh Hiệp định: Là sinh viên nước ngoài được tiếp nhận học tập dài hạn tại ĐHBK Hà Nội theo các Hiệp định, thoả thuận của Chính phủ Việt Nam với các đối tác nước ngoài;\n2. Lưu học sinh ngoài Hiệp định: Là sinh 

In [9]:

for chunk in chunks:
    metadata = chunk['metadata']
    print(f"Chương {metadata['chapter_number']}: {metadata['chapter_title']}, Mục {metadata['section_number']}: {metadata['section_title']}, Điều {metadata['article_number']}: {metadata['article_title']}")
    print("--------")
    print(chunk['text'].strip())
    print("-"*40)

Chương I: # NHỮNG QUY ĐỊNH CHUNG, Mục None: None, Điều 1: Phạm vi điều chỉnh và đối tượng áp dụng
--------
1. # Phạm vi điều chỉnh

1. Quy định này quy định về trách nhiệm của các đơn vị, cá nhân có liên quan trong Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội hoặc Đại học) đối với công tác quản lý và hỗ trợ người nước ngoài (là người không mang quốc tịch Việt Nam hoặc là người Việt Nam mang hộ chiếu nước ngoài) đến học tập, thực tập, nghiên cứu, trao đổi tại ĐHBK Hà Nội;
2. Quy định này quy định về quyền lợi và trách nhiệm của người nước ngoài trong quá trình học tập, thực tập, tham quan, trao đổi tại ĐHBK Hà Nội.
2. # Đối tượng áp dụng

Quy định này áp dụng đối với người nước ngoài đến học tập, thực tập, nghiên cứu, trao đổi tại ĐHBK Hà Nội (sau đây gọi chung là Lưu học sinh và viết tắt là LHS) và được chia thành các nhóm như sau:

1. Lưu học sinh Hiệp định: Là sinh viên nước ngoài được tiếp nhận học tập dài hạn tại ĐHBK Hà Nội theo các Hiệp định, thoả thuận của Chính phủ

In [10]:
import re
from typing import List, Dict, Any, Tuple

def get_content_between_headings(text: str, heading_pattern: str) -> Tuple[List[Dict[str, Any]], str]:

    pattern = re.compile(heading_pattern, re.DOTALL)
    matches = list(pattern.finditer(text))
    results = []
    remaining_content = text
    
    if matches:
        # Process each heading match
        for i, match in enumerate(matches):
            heading_start = match.start()
            heading_end = match.end()
            
            # Determine content end
            content_end = len(text)
            if i < len(matches) - 1:
                content_end = matches[i+1].start()
            
            # Extract the heading and its content
            full_content = text[heading_start:content_end]
            heading_text = match.group(0)
            
            # Store the result
            results.append({
                "match": match,
                "full_text": full_content,
                "heading": heading_text
            })
        
        # Determine remaining content (text before the first heading)
        if matches[0].start() > 0:
            remaining_content = text[:matches[0].start()]
        else:
            remaining_content = ""
    
    return results, remaining_content

def extract_sections_new(text: str) -> List[Dict[str, Any]]:

    # Get all section headings and content
    section_pattern = r'#\s*(\d+\.\d+)\s+([^\n]+)'
    section_matches, remaining_text = get_content_between_headings(text, section_pattern)
    
    sections = []
    for section_data in section_matches:
        match = section_data["match"]
        section_num = match.group(1)
        section_title = match.group(2).strip() if match.group(2) else ""
        full_text = section_data["full_text"]
        heading_text = section_data["heading"]
        
        # Extract the section content (excluding the heading)
        section_content = full_text[len(heading_text):].strip()
        
        sections.append({
            "number": section_num,
            "title": section_title,
            "content": section_content
        })
    
    return sections

def extract_chapters_new(text: str) -> List[Dict[str, Any]]:

    # Get all chapter headings and content
    # This pattern matches both "# I. Title" and "# I- Title" formats
    chapter_pattern = r'#\s*([IVXLCDM]+)(?:\.|-)\s+([^\n]+)'
    chapter_matches, remaining_text = get_content_between_headings(text, chapter_pattern)
    
    chapters = []
    for chapter_data in chapter_matches:
        match = chapter_data["match"]
        chapter_num = match.group(1)
        chapter_title = match.group(2).strip() if match.group(2) else ""
        full_text = chapter_data["full_text"]
        heading_text = chapter_data["heading"]
        
        # Extract the chapter content (excluding the heading)
        chapter_content = full_text[len(heading_text):].strip()
        
        # Extract sections in this chapter
        chapter_sections = extract_sections_new(full_text)
        
        # Calculate content directly belonging to the chapter (before any sections)
        direct_content = ""
        if chapter_sections:
            # Find where the first section starts
            first_section_pattern = r'#\s*\d+\.\d+'
            first_section_match = re.search(first_section_pattern, chapter_content)
            if first_section_match:
                direct_content = chapter_content[:first_section_match.start()].strip()
            else:
                direct_content = chapter_content
        else:
            direct_content = chapter_content
        
        chapters.append({
            "number": chapter_num,
            "title": chapter_title,
            "content": direct_content,
            "sections": chapter_sections
        })
    
    return chapters

def parse_document_new(text: str) -> Dict[str, Any]:
  
    # Extract all structural elements
    chapters = extract_chapters_new(text)
    
    return {
        "chapters": chapters
    }

def get_chunks_with_metadata_new(text: str) -> List[Dict[str, Any]]:

    parsed_doc = parse_document(text)
    chunks = []
    
    # Process chapters
    for chapter in parsed_doc["chapters"]:
        chapter_num = chapter["number"]
        chapter_title = chapter["title"]
        
        # Create a chunk for direct chapter content (if not empty)
        if chapter["content"]:
            chunks.append({
                "text": chapter["content"],
                "metadata": {
                    "chapter_number": chapter_num,
                    "chapter_title": chapter_title,
                    "section_number": None,
                    "section_title": None
                }
            })
        
        # Process sections in chapter
        for section in chapter["sections"]:
            section_num = section["number"]
            section_title = section["title"]
            
            chunks.append({
                "text": section["content"],
                "metadata": {
                    "chapter_number": chapter_num,
                    "chapter_title": chapter_title,
                    "section_number": section_num,
                    "section_title": section_title
                }
            })
    
    return chunks

In [11]:
for chunk in chunks:
    metadata = chunk['metadata']
    print(f"Chương {metadata['chapter_number']}: {metadata['chapter_title']}, Mục {metadata['section_number']}: {metadata['section_title']}")
    print("--------")
    print(chunk['text'])
    print("-"*40)

Chương I: # NHỮNG QUY ĐỊNH CHUNG, Mục None: None
--------
1. # Phạm vi điều chỉnh

1. Quy định này quy định về trách nhiệm của các đơn vị, cá nhân có liên quan trong Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội hoặc Đại học) đối với công tác quản lý và hỗ trợ người nước ngoài (là người không mang quốc tịch Việt Nam hoặc là người Việt Nam mang hộ chiếu nước ngoài) đến học tập, thực tập, nghiên cứu, trao đổi tại ĐHBK Hà Nội;
2. Quy định này quy định về quyền lợi và trách nhiệm của người nước ngoài trong quá trình học tập, thực tập, tham quan, trao đổi tại ĐHBK Hà Nội.
2. # Đối tượng áp dụng

Quy định này áp dụng đối với người nước ngoài đến học tập, thực tập, nghiên cứu, trao đổi tại ĐHBK Hà Nội (sau đây gọi chung là Lưu học sinh và viết tắt là LHS) và được chia thành các nhóm như sau:

1. Lưu học sinh Hiệp định: Là sinh viên nước ngoài được tiếp nhận học tập dài hạn tại ĐHBK Hà Nội theo các Hiệp định, thoả thuận của Chính phủ Việt Nam với các đối tác nước ngoài;
2. Lưu học 

In [12]:
from langchain_core.documents import Document

md_dir = "../data/parse/markdown_v2/"

def format_metadata(metadata: Dict):
    section_mapping = {
        0: "Chương",
        1: "Mục",
        2: "Điều"
    }

    source_mapping = {
        "QC": "Quy chế",
        "QĐ": "Quy định",
        "HD": "Hướng dẫn",
        "QtĐ": "Quyết định"
    }

    source = metadata.get("source").split(".md")[0]

    for k, v in source_mapping.items():
        if source.startswith(k):
            source = source.replace(k, v).upper()

    numbers = (metadata['chapter_number'], metadata['section_number'], metadata['article_number'])
    titles = (metadata['chapter_title'], metadata['section_title'], metadata['article_title'])
    
    formatted_metadata = f"{source}\n"
    for i in range(len(titles)):

        if titles[i] is not None:
            formatted_metadata += f"{section_mapping[i]} {numbers[i]}: {titles[i].upper().strip("# ")}\n\n"
    
    return formatted_metadata

def split_md(dir, **kwargs):
    doc_list = []
    for file in os.listdir(dir):
        full_path = os.path.join(md_dir, file)
        with open(full_path, "r") as f:
            content = f.read()
        if file.startswith("HD"):
            chunks = get_chunks_with_metadata_new(content)
        else:
            chunks = get_chunks_with_metadata(content)

        for chunk in chunks:
            chunk_metadata = chunk["metadata"]
            chunk_metadata["source"] = file
            formatted_metadata = format_metadata(chunk_metadata)
            doc_list.append(Document(page_content=formatted_metadata + chunk["text"], metadata=chunk_metadata, **kwargs))

    return doc_list

doc_list = split_md(md_dir)

In [13]:
print(doc_list[130])

page_content='QUY CHẾ TUYỂN SINH ĐH 2023
Chương I: QUY ĐỊNH CHUNG

Điều 6: CÁC BAN CHUYÊN MÔN CỦA HỘI ĐỒNG TUYỂN SINH

1. Các ban chuyên môn của HĐTS bao gồm Ban Thư ký HĐTS, Ban chỉ đạo Kỳ thi ĐGTD, Hội đồng thi ĐGTD, Ban xét tuyển ĐHCQ theo hồ sơ năng lực, Ban xét tuyển ĐHCQ theo kết quả thi, Ban xét tuyển đại học hình thức VLVH, được Giám đốc đại học ra quyết định thành lập.
2. Ban thư ký HĐTS có nhiệm vụ giúp Hội đồng tuyển sinh thực hiện các công việc: công bố Đề án tuyển sinh và các thông tin tuyển sinh khác trên Cổng thông tin tuyển sinh; dự thảo kết quả tuyển sinh (điểm chuẩn trúng tuyển và danh sách trúng tuyển); dự thảo các báo cáo về kết quả tuyển sinh trình Chủ tịch HĐTS và các nhiệm vụ khác do Chủ tịch HĐTS giao.
3. Ban chỉ đạo Kỳ thi ĐGTD có nhiệm vụ chỉ đạo tổ chức Kỳ thi, các công tác liên quan, quyết định các tình huống đặc biệt; kiểm tra việc thực hiện nhiệm vụ của Hội đồng thi và các ban; xử lý các vấn đề phát sinh trong quá trình tổ chức Kỳ thi.
4. Ban xét tuyển ĐHC

In [14]:
json_list = []
with open("../data/chunks/chunks_v2.json", "w", encoding="utf-8") as f:
    for idx, doc in enumerate(doc_list):
        doc_content = {"id": idx, "page_content": doc.page_content, "metadata": doc.metadata}
        json_list.append(doc_content)
    content = json.dumps(json_list, indent=4, ensure_ascii=False)
    f.write(content)

In [17]:
import json
import tiktoken
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter

In [23]:
def analyze_tokens(json_file_path):
    # Load the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Initialize the tiktoken encoder
    # Using 'cl100k_base' which is used by newer models like gpt-4, gpt-3.5-turbo
    enc = tiktoken.get_encoding("o200k_base")
    
    # Process each entry and tokenize
    token_counts = []
    all_tokens = []
    
    for item in data:
        page_content = item.get("page_content", "")
        
        # Tokenize the text
        tokens = enc.encode(page_content)
        
        # Store the token count
        token_counts.append(len(tokens))
        
        # Add to all tokens list for vocabulary analysis
        all_tokens.extend(tokens)
    
    # Calculate statistics
    total_tokens = sum(token_counts)
    avg_tokens = np.mean(token_counts)
    median_tokens = np.median(token_counts)
    max_tokens = np.max(token_counts)
    min_tokens = np.min(token_counts)
    
    print(f"Total number of entries: {len(data)}")
    print(f"Total tokens across all entries: {total_tokens}")
    print(f"Average tokens per entry: {avg_tokens:.2f}")
    print(f"Median tokens per entry: {median_tokens}")
    print(f"Maximum tokens in an entry: {max_tokens}")
    print(f"Minimum tokens in an entry: {min_tokens}")
    
    # Create a separate histogram chart for token distribution
    plt.figure(figsize=(12, 6))
    plt.hist(token_counts, bins=50, alpha=0.7, color='teal', edgecolor='black')
    plt.axvline(avg_tokens, color='r', linestyle='dashed', linewidth=1, label=f'Mean: {avg_tokens:.2f}')
    plt.axvline(median_tokens, color='g', linestyle='dashed', linewidth=1, label=f'Median: {median_tokens}')
    plt.title('Distribution of Token Counts per Entry', fontsize=14)
    plt.xlabel('Number of Tokens', fontsize=12)
    plt.ylabel('Number of Entries', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.savefig('token_distribution_histogram.png')
    plt.close()
    
    # Create a separate chart for token frequency analysis
    token_freq = Counter(all_tokens)
    top_tokens = token_freq.most_common(50)
    
    # Convert token IDs to actual tokens for readability
    top_token_words = [(enc.decode([t[0]]), t[1]) for t in top_tokens]
    
    # Create a dataframe for plotting
    df = pd.DataFrame(top_token_words, columns=['Token', 'Frequency'])
    
    # Plot the token frequency
    plt.figure(figsize=(14, 7))
    bars = plt.bar(range(len(df)), df['Frequency'], color='teal', alpha=0.7, edgecolor='black')
    plt.xticks(range(len(df)), df['Token'], rotation=90, fontsize=8)
    plt.title('Top 50 Most Frequent Tokens', fontsize=14)
    plt.xlabel('Token', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('token_frequency_chart.png')
    plt.close()
    
    return {
        'total_tokens': total_tokens,
        'avg_tokens': avg_tokens,
        'token_counts': token_counts,
        'token_frequencies': token_freq
    }

if __name__ == "__main__":
    # Replace with your JSON file path
    json_file_path = "../data/chunks/chunks_v2.json"
    results = analyze_tokens(json_file_path)

Total number of entries: 471
Total tokens across all entries: 211749
Average tokens per entry: 449.57
Median tokens per entry: 315.0
Maximum tokens in an entry: 3032
Minimum tokens in an entry: 47


{'Hust_doc_final': _CollectionConfigSimple(name='Hust_doc_final', description=None, generative_config=None, properties=[_Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none')], references=[], reranker_config=None, vectorizer_config=None, vectorizer=<Vectorizers.NONE: 'none'>, vector_config=None),
 'Hust_doc_text': _CollectionConfigSimple(name='Hust_doc_text', description=None, generative_config=None, properties=[_Property(name='text', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=None, vectorizer='none'), _Property(name='source', description="This property was generated by Weaviate's auto-schema feature on Wed Mar 19 15:26:55